<a href="https://colab.research.google.com/github/GalindoD/Backtesting-v1/blob/main/Backtester_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install backtrader yfinance matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 7.8 MB/s eta 0:00:00


In [2]:

import pandas as pd
import numpy as np
import plotly.express as px
import backtrader as bt
import yfinance as yf
import matplotlib.pyplot as plt
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda
!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4083    0  4083    0     0  18956      0 --:--:-- --:--:-- --:--:-- 18902
100  517k  100  517k    0     0   744k      0 --:--:-- --:--:-- --:--:--  744k
--2025-04-23 14:52:36--  https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
Resolving anaconda.org (anaconda.org)... 104.19.145.37, 104.19.144.37, 2606:4700::6813:9025, ...
Connecting to anaconda.org (anaconda.org)|104.19.145.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5f738b4b78d4370a69f82984/673b62920c609ff6945bb24d?response-content-disposition=attachment%3B%20filename%3D%22ta-lib-0.5.1-py311h9ecbd09_0.conda%22%3B%20filename%2A%3DUTF-8%27%27ta-lib-0.5.1-py311h9ecbd09_0.conda&response-content-type=application%2Foctet-stream&X-Amz-Algorithm=

#Function for Data gathering

In [64]:
def gatherdata(ticker, periodicity):
  data = yf.download(
      tickers = ticker,
      #start="2022-12-31",
      #end="2025-04-19",
      #period="max",
      interval=periodicity,
      ignore_tz=True,
      auto_adjust=True)
  return data

def createdf(data, fastma, slowma, fastmacd, slowmacd, signal):
  data_df = data.copy()
  data_df.columns = ["close", "high", "low", "open", "volume"]
  data_df.drop(columns=["volume", "high", "low", "open"], inplace=True)

  data_df["R"] = data_df.close.pct_change().fillna(0)

  #Calculating values of Indicators
  data_df["slow_ma"] = talib.SMA(data_df.close, slowma)
  data_df["fast_ma"] = talib.SMA(data_df.close, fastma)
  data_df["macd"], data_df["signal"], data_df["histogram"] = talib.MACD(data_df.close, fastperiod=fastmacd, slowperiod=slowmacd, signalperiod=signal)

  #Delete First NA rows, resultring form lack of previous data
  data_df.dropna(inplace=True)
  return data_df

#Function for Computing Buy Signals

In [65]:

def signalcompute(data_df):
  data_df = data_df.assign(
      macdbuy = lambda x: np.where( x.macd > x.signal ,1,0)
      )
  data_df = data_df.assign(
      fastmabuy = lambda x: np.where( x.close > x.fast_ma ,1,0)
      )
  data_df = data_df.assign(
      slowmabuy = lambda x: np.where( x.close > x.slow_ma ,1,0)
      )
  return data_df

In [66]:
def compute_strategy(data_df, strategy, days, liquid_return):
  data_df["buy"] = 0
  match strategy:
    case 1:
      #Strategy 1: MACD and MA, buy both signals 1 and sell both signals 0
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"]  == 1):
          data_df.loc[i, "buy"] = 1
        elif ((data_df.loc[i, "macdbuy"] == 1 or data_df.loc[i, "fastmabuy"] == 1) and data_df.loc[i - pd.Timedelta(days=days), "buy"] == 1):
          data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0
    case 2:
      #Strategy 2: MACD and fastMA, buy both signals 1 and sell when either signal is 0
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"] == 1):
            data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0
    case 3:
      #Strategy 3: Only MACD
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 ):
            data_df.loc[i, "buy"] = 1
        else:
            data_df.loc[i, "buy"] = 0
    case 4:
      #Strategy 4: MACD and SlowMA and FastMA
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"] == 1 and data_df.loc[i, "slowmabuy"] == 1):
          data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0

  #Shift
  data_df["buy"] = data_df["buy"].shift(1, fill_value=0)

  #Buying according to strategy
  data_df["R_strategy"] = data_df.R * data_df.buy

  #Assuming 5% return on liquid
  data_df['R_strategy'] = data_df['R_strategy'].replace([0], liquid_return)

  #Calculating total Return
  #returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1

  #Adding transaction costs to Strategy
  for i, row in data_df.iterrows():
      try:
        if data_df.loc[i, "buy"] != data_df.loc[i + pd.Timedelta(days=days), "buy"]:
          data_df.loc[i + pd.Timedelta(days=days), "R_strategy"] = data_df.loc[i + pd.Timedelta(days=days), "R_strategy"] - trsncost
        else:
          continue
      except:
        print("End")
  return data_df


In [67]:
#Main Cell


#Variables
ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.000484
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

#Strategy 1: MACD and FastMA, buy both signals 1 and sell both signals 0. | 5d: 0.658x | 1wk: 0.5089|
#Strategy 2: MACD and FastMA, buy both signals 1 and sell when either signal is 0. | 5d: 0.8033x | 1wk: 0.5063|
#Strategy 3: Only MACD. | 5d: 0.8409x | 1wk:  0.5176|
#Strategy 4: MACD and 40SlowMA and FastMA. | 5d: 0.9272 | 1wk: 0.5263 |
strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9 --- OPtimized: Slowmacd = 19
fastmacd = 8
slowmacd = 19
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy, days, liquid_return)




[*********************100%***********************]  1 of 1 completed


End


In [68]:
#Total Return with transaction costs
returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1
print("With transaction cost:")
print("Return of Holding: ", returns.iloc[0])
print("Return of Strategy: ", returns.iloc[1])
print("The return of this strategy is ",round((returns.iloc[1]/returns.iloc[0]).item(), 4), "x of holding. Ticker: ", ticker)

With transaction cost:
Return of Holding:  37701.20016916965
Return of Strategy:  38376.651887957734
The return of this strategy is  1.0179 x of holding. Ticker:  BTC-USD


In [69]:
px.line(100 * (1 + data_df[["R","R_strategy"]]).cumprod(), title=f"Total Return: {ticker} {periodicity} -> FMCD: {fastmacd} SMCD: {slowmacd} SIG: {signal}.")

In [70]:
data_df.tail(20)

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,R_strategy
Date,,,,,,,,,,,,
2025-01-17,104462.039062,0.105554,75085.021973,98134.920313,7080.558779,7838.840051,-758.281273,0,1,1,0,0.000484
2025-01-22,103653.070312,-0.007744,76218.760254,98900.010938,7140.701961,7699.212433,-558.510473,0,1,1,0,0.000484
2025-01-27,102087.687500,-0.015102,77337.329590,98985.178125,6832.795356,7525.929018,-693.133662,0,1,1,0,0.000484
2025-02-01,100655.906250,-0.014025,78226.298535,98904.842969,6290.425350,7278.828284,-988.402934,0,1,1,0,0.000484
2025-02-06,96593.296875,-0.040361,78937.267871,98560.018750,5274.437970,6877.950222,-1603.512251,0,0,1,0,0.000484
2025-02-11,95747.429688,-0.008757,79633.152051,98666.137500,4342.653304,6370.890838,-2028.237534,0,0,1,0,0.000484
2025-02-16,96175.031250,0.004466,80422.046582,98767.247656,3646.163044,5825.945279,-2179.782235,0,0,1,0,0.000484
2025-02-21,96125.546875,-0.000515,81475.398828,98691.114844,3071.545927,5275.065409,-2203.519482,0,0,1,0,0.000484
2025-02-26,84347.023438,-0.122533,82060.429102,97433.546875,1161.460059,4452.344339,-3290.884280,0,0,1,0,0.000484


#Plot last 200 values

In [71]:
last_200 = data_df.tail(200)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 2 rows, 1 column, shared x-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    #subplot_titles=("Price and Moving Averages", "MACD")
                    )

# Row 1: Price and Moving Averages
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['close'],
                         mode='lines', name='Close'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['slow_ma'],
                         mode='lines', name='SlowMA'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['fast_ma'],
                         mode='lines', name='FastMA'),
              row=1, col=1)

# Row 2: MACD and Signal Line
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['macd'],
                         mode='lines', name='MACD'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['signal'],
                         mode='lines', name='Signal'),
              row=2, col=1)

# Horizontal line at y=0 on MACD plot
fig.add_shape(type="line",
              x0=last_200.index.min(), x1=last_200.index.max(),
              y0=0, y1=0,
              line=dict(color="gray", width=2),
              row=2, col=1)

# Layout and labels
fig.update_layout(height=600, width=900, showlegend=True,
                  title_text=f"Price, MA, MACD: {ticker}.")

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="MACD", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

fig.show()

In [72]:
data_df

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,R_strategy
Date,,,,,,,,,,,,
2015-03-31,244.223999,-0.017334,313.141575,260.901698,-11.506411,-16.736175,5.229764,1,0,0,0,0.000484
2015-04-05,260.597992,0.067045,308.223174,261.229396,-10.140698,-15.417080,5.276381,1,0,0,0,0.000484
2015-04-10,236.072006,-0.094114,304.071174,260.808296,-11.956971,-14.725058,2.768087,1,0,0,0,0.000484
2015-04-15,223.832993,-0.051844,299.678999,259.318095,-14.458531,-14.671753,0.213222,1,0,0,0,0.000484
2015-04-20,224.626007,0.003543,295.917850,255.760497,-15.791398,-14.895682,-0.895717,0,0,0,0,0.000484
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-02,82485.710938,-0.022138,86407.344043,86554.630469,-2805.039310,-819.081185,-1985.958124,0,0,0,0,0.000484
2025-04-07,79235.335938,-0.039405,86814.716016,84860.660937,-3312.480657,-1317.761080,-1994.719577,0,0,0,0,0.000484
2025-04-12,85287.109375,0.076377,87339.344531,83776.817187,-2854.417721,-1625.092408,-1229.325313,0,1,0,0,0.000484


In [31]:
#Loop Cell la buena


#Variables
ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.000484
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9
fastmacd = 8
slowmacd = 17
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

row_name = "slowmacd_" + str(slowmacd)
column_name = "fastmacd_" + str(fastmacd)

returns_df = pd.DataFrame({column_name: [0]}, index=[row_name])

for slowmacd in range(17, 29):
  row_name = "slowmacd_" + str(slowmacd)
  #returns_df.loc[row_name, column_name] = pd.DataFrame({'Values': [row_name]})

  for fastmacd in range(7, 15):
    #Create dataframe
    data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

    #Compute the signal
    data_df = signalcompute(data_df)

    #Choose and compute strategy
    data_df = compute_strategy(data_df, strategy, days, liquid_return)

    #Total Return with transaction costs
    retvals = (100 * (1+data_df[["R_strategy"]]).prod()-1).item()
    column_name = "fastmacd_" + str(fastmacd)
    returns_df.loc[row_name, column_name] =  retvals

[*********************100%***********************]  1 of 1 completed


End
End


<ipython-input-31-c9a84258d6c4>:59: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '36798.476698017665' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End
End


KeyboardInterrupt: 

In [ ]:
returns_df

In [ ]:
fig = px.imshow(returns_df, text_auto=True, aspect="auto")
fig.show()

In [73]:
#Main Cell


#Variables
#ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"
ticker = "SOL-USD"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.000684
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

#Strategy 1: MACD and FastMA, buy both signals 1 and sell both signals 0. | 5d: 0.658x | 1wk: 0.5089|
#Strategy 2: MACD and FastMA, buy both signals 1 and sell when either signal is 0. | 5d: 0.8033x | 1wk: 0.5063|
#Strategy 3: Only MACD. | 5d: 0.8409x | 1wk:  0.5176|
#Strategy 4: MACD and 40SlowMA and FastMA. | 5d: 0.9272 | 1wk: 0.5263 |
strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9
fastmacd = 8
slowmacd = 19
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy, days, liquid_return)




[*********************100%***********************]  1 of 1 completed

End


In [74]:
#Total Return with transaction costs
returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1
print("With transaction cost:")
print("Return of Holding: ", returns.iloc[0])
print("Return of Strategy: ", returns.iloc[1])
print("The return of this strategy is ",round((returns.iloc[1]/returns.iloc[0]).item(), 4), "x of holding. Ticker: ", ticker)

With transaction cost:
Return of Holding:  6878.976489948947
Return of Strategy:  21643.644728348867
The return of this strategy is  3.1463 x of holding. Ticker:  SOL-USD


In [75]:
px.line(100 * (1 + data_df[["R","R_strategy"]]).cumprod(), title=f"Total Return: {ticker} {periodicity} -> FMCD: {fastmacd} SMCD: {slowmacd} SIG: {signal}.")

In [80]:
data_df.head(20)

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,R_strategy
Date,,,,,,,,,,,,
2020-10-22,1.955653,-0.111811,1.679599,2.688397,0.050273,0.336177,-0.285904,0,0,1,0,0.000684
2020-10-27,1.710232,-0.125493,1.698578,2.566568,-0.050222,0.258897,-0.309119,0,0,1,0,0.000684
2020-11-01,1.599840,-0.064548,1.722408,2.393286,-0.132945,0.180529,-0.313473,0,0,0,0,0.000684
2020-11-06,1.913740,0.196207,1.756781,2.295472,-0.149531,0.114517,-0.264047,0,0,1,0,0.000684
2020-11-11,2.002125,0.046184,1.790751,2.204133,-0.147015,0.062210,-0.209226,0,0,1,0,0.000684
2020-11-16,2.255355,0.126481,1.829859,2.114374,-0.111037,0.027561,-0.138598,0,1,1,0,0.000684
2020-11-21,2.382205,0.056244,1.873368,2.089986,-0.067881,0.008472,-0.076354,0,1,1,0,0.000684
2020-11-26,1.903037,-0.201145,1.907518,2.051110,-0.094728,-0.012168,-0.082561,0,0,0,0,0.000684
2020-12-01,1.976308,0.038502,1.942932,1.990034,-0.102461,-0.030226,-0.072235,0,0,1,0,0.000684


In [78]:
last_200 = data_df.tail(200)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 2 rows, 1 column, shared x-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    #subplot_titles=("Price and Moving Averages", "MACD")
                    )

# Row 1: Price and Moving Averages
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['close'],
                         mode='lines', name='Close'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['slow_ma'],
                         mode='lines', name='SlowMA'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['fast_ma'],
                         mode='lines', name='FastMA'),
              row=1, col=1)

# Row 2: MACD and Signal Line
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['macd'],
                         mode='lines', name='MACD'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['signal'],
                         mode='lines', name='Signal'),
              row=2, col=1)

# Horizontal line at y=0 on MACD plot
fig.add_shape(type="line",
              x0=last_200.index.min(), x1=last_200.index.max(),
              y0=0, y1=0,
              line=dict(color="gray", width=2),
              row=2, col=1)

# Layout and labels
fig.update_layout(height=600, width=900, showlegend=True,
                  title_text=f"Price, MA, MACD: {ticker}.")

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="MACD", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

fig.show()

#Test code